In [ ]:
import pandas as pd
import ast
import os,sys

# --- Read File Selection ---
try:
    read_file = input("Enter read file path (e.g., raw_stream/raw_data_x.csv): ").strip()
    if not read_file or not os.path.isfile(read_file):
        raise FileNotFoundError(f"Read file does not exist: {read_file}")
except Exception as e:
    print(f"[ERROR] {e}")
    sys.exit(1)

# --- Write File Selection ---
try:
    write_file = input("Enter write file path (e.g., processed_stream/processed_data_x.csv): ").strip()
    if not write_file:
        raise ValueError("No write file path provided.")
    write_dir = os.path.dirname(write_file) or "."
    os.makedirs(write_dir, exist_ok=True)  # create parent dirs if needed
    if os.path.exists(write_file):
        os.remove(write_file)
        print(f"[INFO] Existing file '{write_file}' deleted.")
except Exception as e:
    print(f"[ERROR] {e}")
    sys.exit(1)

# --- Processing Logic ---
frames = []

for chunk in pd.read_csv(read_file, usecols=["times", "LAT", "LON", "PM2.5", "PM10", "O3", ], dtype=str, chunksize=50_000):
    chunk = chunk[(chunk['times'] != 'times') & (chunk["LAT"] != "LAT") & (chunk["LON"] != "LON") & (chunk["PM2.5"] != "PM2.5") & (chunk["PM10"] != "PM10") & (chunk["O3"] != "O3")].reset_index(drop=True)
    chunk['times'] = chunk['times'].apply(ast.literal_eval)
    chunk['LAT'] = chunk['LAT'].apply(ast.literal_eval)
    chunk['LON'] = chunk['LON'].apply(ast.literal_eval)
    chunk['PM2.5'] = chunk['PM2.5'].apply(ast.literal_eval)
    chunk['PM10'] = chunk['PM10'].apply(ast.literal_eval)
    chunk['O3'] = chunk['O3'].apply(ast.literal_eval)
    chunk = chunk.explode(['times', 'LAT', 'LON', 'PM2.5', 'PM10', 'O3'], ignore_index=True)
    chunk['times'] = pd.to_datetime(chunk['times'], errors='coerce')
    chunk['LAT'] = pd.to_numeric(chunk['LAT'], errors='coerce')
    chunk['LON'] = pd.to_numeric(chunk['LON'], errors='coerce')
    chunk['PM2.5'] = pd.to_numeric(chunk['PM2.5'], errors='coerce')
    chunk['PM10'] = pd.to_numeric(chunk['PM10'], errors='coerce')
    chunk['O3'] = pd.to_numeric(chunk['O3'], errors='coerce')
    chunk = chunk.dropna(subset=['times', 'LAT', 'LON', 'PM2.5', 'PM10', 'O3'])
    chunk['hour'] = chunk['times'].dt.floor('h')
    hourly = (
        chunk.groupby(['hour', 'LAT', 'LON'], as_index=False)[['PM2.5', 'PM10', 'O3']]
        .mean()
        .rename(columns={
            'PM2.5': 'pm25_mean',
            'PM10': 'pm10_mean',
            'O3': 'o3_mean'
        })
    )
    frames.append(hourly)

df = pd.concat(frames)
df.to_csv(write_file, index=False)
print(f"Processed data saved to: {write_file}")

In [ ]:
import pandas as pd

file1 = "processed_stream/processed_data_2412.csv"
file2 = "processed_stream/processed_data_2501.csv"
file3 = "processed_stream/processed_data_2502.csv"

# Read each CSV into a DataFrame
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)

# Concatenate all three
df = pd.concat([df1, df2, df3], ignore_index=True)

In [ ]:
dfc = df.copy()

In [ ]:
# PM2.5 Heatmap (matplotlib)

import matplotlib.pyplot as plt
import numpy as np

df['hour'] = pd.to_datetime(df['hour'], errors='coerce')

# split into date + hour-of-day
df['date'] = df['hour'].dt.date
df['hod']  = df['hour'].dt.hour

g = df.groupby(['date','hod'], as_index=False)['pm25_mean'].mean()

heat = (
    g.pivot(index='hod', columns='date', values='pm25_mean')
     .sort_index()
)
# plot (matplotlib, no explicit colors)
fig, ax = plt.subplots(figsize=(14, 6))
im = ax.imshow(heat.to_numpy(), aspect='auto', origin='lower')

ax.set_yticks(np.arange(heat.shape[0]))
ax.set_yticklabels(heat.index)

cols = heat.columns
step = max(1, len(cols)//30)
ax.set_xticks(np.arange(0, len(cols), step))
ax.set_xticklabels([c.strftime('%Y-%m-%d') for c in cols[::step]], rotation=90)

ax.set_xlabel('Date'); ax.set_ylabel('Hour of day')
ax.set_title('PM2.5 hourly mean (hour × day)')
fig.colorbar(im, ax=ax, label='µg/m³')
fig.tight_layout(); plt.show()


In [ ]:
# PM2.5 Heatmap (interactive, plotly)

import numpy as np
import pandas as pd
import plotly.express as px

# --- 1) Ensure df['hour'] is datetime ---
# If it's already datetime, this is a no-op. If it's a string/number, it will parse it.
df = dfc.copy()
df['hour'] = pd.to_datetime(df['hour'], errors='coerce')

# Drop rows that failed to parse
df = df.dropna(subset=['hour'])

# --- 2) Derive date and hour-of-day ---
df['date'] = df['hour'].dt.normalize()     # midnight for each date (datetime64[ns])
df['hod']  = df['hour'].dt.hour            # 0..23

# --- 3) Build pivot: rows = hour-of-day, cols = date, values = pm25_mean ---
heat = (
    df.pivot_table(index='hod', columns='date', values='pm25_mean', aggfunc='mean')
      .sort_index()
)

# If columns are date objects (rare), convert to datetime for nicer axis formatting
if not np.issubdtype(heat.columns.dtype, np.datetime64):
    heat.columns = pd.to_datetime(heat.columns)

# --- 4) Classify danger levels (heuristic) ---
bins  = [-np.inf, 12.0, 35.4, 55.4, 150.4, np.inf]
names = ["Good", "Moderate", "USG", "Unhealthy", "Very Unhealthy"]

vals = heat.values
cat_idx = np.digitize(vals, bins) - 1
cat_idx = np.clip(cat_idx, 0, len(names)-1)

custom = np.where(
    np.isnan(vals),
    "No data",
    np.array(names, dtype=object)[cat_idx]
)

# --- 5) Interactive heatmap ---
fig = px.imshow(
    heat,
    origin="lower",
    aspect="auto",
    labels=dict(x="Date", y="Hour of day", color="µg/m³"),
    x=heat.columns, y=heat.index,
    color_continuous_scale="Viridis",
)

# Attach 2-D customdata matching z-shape
fig.update_traces(
    customdata=custom,
    hovertemplate="Date=%{x|%Y-%m-%d}<br>Hour=%{y}:00<br>PM2.5=%{z:.1f} µg/m³<br>Status=%{customdata}<extra></extra>"
)

fig.update_layout(
    title="PM2.5 hourly mean (interactive heatmap)",
    xaxis_tickangle=-45,
    yaxis_nticks=24,
)
fig.show()


In [ ]:
# PM2.5 Levels and Increments Over Time (interactive, plotly)

df = dfc.copy()
df['hour'] = pd.to_datetime(df['hour'], errors='coerce')
df = df.sort_values('hour')
df['pm25_delta'] = df['pm25_mean'].diff()

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['hour'],
    y=df['pm25_mean'],
    mode='lines+markers',
    name='PM2.5',
    line=dict(color='blue')
))
fig.add_trace(go.Scatter(
    x=df['hour'],
    y=df['pm25_delta'],
    mode='lines',
    name='Δ PM2.5',
    line=dict(color='red', dash='dot')
))

fig.update_layout(
    title="PM2.5 Levels and Increments Over Time",
    xaxis_title="Timestamp",
    yaxis_title="µg/m³",
    hovermode="x unified"
)
fig.show()


In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go

df = dfc.copy()

fig = sp.make_subplots(rows=3, cols=1, shared_xaxes=True,
                       subplot_titles=["PM2.5", "PM10", "O₃"],
                       vertical_spacing=0.05)

fig.add_trace(go.Scatter(x=df['hour'], y=df['pm25_mean'],
                         mode='lines+markers', name='PM2.5',
                         line=dict(color='indigo')), row=1, col=1)

fig.add_trace(go.Scatter(x=df['hour'], y=df['pm10_mean'],
                         mode='lines+markers', name='PM10',
                         line=dict(color='crimson')), row=2, col=1)

fig.add_trace(go.Scatter(x=df['hour'], y=df['o3_mean'],
                         mode='lines+markers', name='O₃',
                         line=dict(color='teal')), row=3, col=1)

fig.update_layout(height=600, width=1200, title="Pollutant Trends Over Time",
                  xaxis3_title="Timestamp", yaxis_title="µg/m³",
                  hovermode="x unified", template="plotly_white")
fig.show()


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px


df = dfc.copy()
df['hour'] = pd.to_datetime(df['hour'], errors='coerce')
df = df.dropna(subset=['hour','LAT','LON','pm25_mean'])


BIN_DEG = 0.01  # ~100 m -> 0.001; tweak larger if you have many near-duplicates
def bin_coord(x, step): return np.floor(x/step)*step
df['lat_bin'] = bin_coord(df['LAT'], BIN_DEG)
df['lon_bin'] = bin_coord(df['LON'], BIN_DEG)
df['loc'] = df['lat_bin'].round(5).astype(str) + "," + df['lon_bin'].round(5).astype(str)


df['date'] = df['hour'].dt.date
df['tod_hours'] = df['hour'].dt.hour + df['hour'].dt.minute/60.0


top9 = (df.groupby('loc').size().sort_values(ascending=False).head(9).index)
plot_df = df[df['loc'].isin(top9)].copy()


cmin = float(plot_df['pm25_mean'].min())
cmax = float(plot_df['pm25_mean'].max())


fig = px.scatter(
    plot_df,
    x='date', y='tod_hours',
    color='pm25_mean', range_color=[cmin, cmax],
    color_continuous_scale='Turbo',
    size='pm25_mean', size_max=16, opacity=0.9,
    facet_col='loc', facet_col_wrap=3,
    hover_data={'LAT':':.5f','LON':':.5f','pm25_mean':':.2f','tod_hours':':.2f','date':True},
    labels={'date':'Date','tod_hours':'Time of day (hours)','pm25_mean':'PM2.5 (µg/m³)'},
    title='Hourly PM2.5 by Location — marker color & size reflect PM2.5'
)


fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))


fig.update_yaxes(range=[0, 24], dtick=2, title='Time of day (hours)')
fig.update_xaxes(title='Date')


fig.update_layout(
    hovermode='closest',
    coloraxis_colorbar_title='PM2.5 (µg/m³)',
    margin=dict(l=40,r=10,t=60,b=40),
    height=800
)

fig.show()


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import os

# ---- Start from your processed df (hourly-ish raw), ensure hour is datetime ----
df = dfc.copy()
df['hour'] = pd.to_datetime(df['hour'], errors='coerce')
df = df.dropna(subset=['hour','LAT','LON','pm25_mean'])

# ---- Define "location" (light binning to merge nearly-identical coords) ----
BIN_DEG = 0.01  # ~1km-ish; tweak as needed
def bin_coord(x, step): return np.floor(x/step)*step
df['lat_bin'] = bin_coord(df['LAT'], BIN_DEG)
df['lon_bin'] = bin_coord(df['LON'], BIN_DEG)
df['loc'] = df['lat_bin'].round(5).astype(str) + "," + df['lon_bin'].round(5).astype(str)

# =====================================================================
# NEW: Aggregate to ONE row per (loc, hour) -> hourly mean per location
# =====================================================================
agg = (
    df.groupby(['loc','lat_bin','lon_bin','hour'], as_index=False)
      .agg(
          pm25_mean=('pm25_mean','mean'),   # average per location-hour
          LAT=('LAT','mean'),               # representative lat of cell
          LON=('LON','mean'),               # representative lon of cell
          n_obs=('pm25_mean','size')        # how many raw points in that loc-hour
      )
)

# ---- Build date & time-of-day for plotting ----
agg['date'] = agg['hour'].dt.date
agg['tod_hours'] = agg['hour'].dt.hour + agg['hour'].dt.minute/60.0

# ---- Pick top locations to keep the chart readable (change to 6 if you want exactly six) ----
TOP_N = 9
top_locs = agg.groupby('loc')['n_obs'].sum().sort_values(ascending=False).head(TOP_N).index
plot_df = agg[agg['loc'].isin(top_locs)].copy()

# ---- Sort so higher PM overlays lower (purely visual) ----
plot_df = plot_df.sort_values(['loc','date','tod_hours','pm25_mean'])

# ---- Consistent color scale across facets ----
cmin = float(plot_df['pm25_mean'].min())
cmax = float(plot_df['pm25_mean'].max())

# ---- Plot: one facet per location, x=date, y=time-of-day, color/size=PM2.5 ----
fig = px.scatter(
    plot_df,
    x='date', y='tod_hours',
    color='pm25_mean', range_color=[cmin, cmax],
    color_continuous_scale='Turbo',
    size='pm25_mean', size_max=16, opacity=0.75,
    facet_col='loc', facet_col_wrap=3,
    hover_data={'LAT':':.5f','LON':':.5f','pm25_mean':':.2f',
                'n_obs':True, 'tod_hours':':.2f','date':True},
    labels={'date':'Date','tod_hours':'Time of day (hours)','pm25_mean':'PM2.5 (µg/m³)'},
    title=f'Hourly mean PM2.5 per location (grid {BIN_DEG}°)'
)

# Tidy facet titles
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

# Axes & layout
fig.update_traces(marker=dict(line=dict(width=0.5)))
fig.update_yaxes(range=[0, 24], dtick=2, title='Time of day (hours)')
fig.update_xaxes(title='Date')
fig.update_layout(
    hovermode='closest',
    coloraxis_colorbar_title='PM2.5 (µg/m³)',
    margin=dict(l=40,r=10,t=60,b=40),
    height=900
)

# fig.show()
fig.show(config={
    "displaylogo": False,
    "toImageButtonOptions": {
        "format": "png",
        "filename": "hourly_pm25_by_location",
        "height": 900,
        "width": 1600,
        "scale": 2
    }
})
